# VacationPy
---



In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,6.07,83,54,0.45,AU,1718136469
1,1,tiksi,71.6872,128.8694,0.92,87,43,0.59,RU,1718136469
2,2,twon-brass,4.3151,6.2418,26.61,82,100,4.49,NG,1718136469
3,3,boa vista,2.8197,-60.6733,31.99,55,40,2.06,BR,1718136469
4,4,lihue,21.9789,-159.3672,28.58,67,0,5.66,US,1718136469


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    tiles = "OSM",
    color = "magenta",
    alpha = 0.5,

    frame_width = 800,
    hover_cols=['City', 'Humidity', 'Max Temp', 'Cloudiness', 'Wind Speed', 'Country'],
    frame_height = 600  
      
)

# Display the map plot
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Max Temp,Cloudiness,Wind Speed,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
ideal_weather_cities = city_data_df[
    (city_data_df['Max Temp'] >= 18) & (city_data_df['Max Temp'] <= 29) &
    (city_data_df['Humidity'] >= 48) & (city_data_df['Humidity'] <= 95) &
    (city_data_df['Wind Speed'] <= 15)
]

# Drop any rows with null values
ideal_weather_cities = ideal_weather_cities.dropna()

# Display sample data
ideal_weather_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,twon-brass,4.3151,6.2418,26.61,82,100,4.49,NG,1718136469
4,4,lihue,21.9789,-159.3672,28.58,67,0,5.66,US,1718136469
8,8,kurchatov,51.6601,35.6521,23.23,71,7,4.35,RU,1718136341
10,10,klyavlino,54.2649,52.0267,18.19,80,1,1.78,RU,1718136469
14,14,adamstown,-25.0660,-130.1015,22.73,80,95,6.96,PN,1718136470
...,...,...,...,...,...,...,...,...,...,...
562,562,tsiombe,-25.3000,45.4833,19.34,72,90,6.43,MG,1718136512
563,563,talodi,10.6325,30.3797,25.68,69,98,0.83,SD,1718136512
564,564,hubynykha,48.8108,35.2547,21.56,61,21,3.42,UA,1718136512
565,565,tirano,46.2154,10.1679,18.29,89,98,1.46,IT,1718136512


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:

hotel_df = ideal_weather_cities.copy()
hotel_df = hotel_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df['Hotel Name'] = ""
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
2,twon-brass,NG,4.3151,6.2418,82,
4,lihue,US,21.9789,-159.3672,67,
8,kurchatov,RU,51.6601,35.6521,71,
10,klyavlino,RU,54.2649,52.0267,80,
14,adamstown,PN,-25.0660,-130.1015,80,
...,...,...,...,...,...,...
562,tsiombe,MG,-25.3000,45.4833,72,
563,talodi,SD,10.6325,30.3797,69,
564,hubynykha,UA,48.8108,35.2547,61,
565,tirano,IT,46.2154,10.1679,89,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "format":"json",
    "categories":"accommodation.hotel",
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame

    latitude = row["Lat"] 
    longitude = row["Lng"]


    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty

    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
twon-brass - nearest hotel: KC Hotel
lihue - nearest hotel: Kauai Palms
kurchatov - nearest hotel: Россия
klyavlino - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
bethel - nearest hotel: Hampton Inn Danbury
touros - nearest hotel: Pousada Atlântico
corfu - nearest hotel: Hotel Bella Venezia
utrik - nearest hotel: No hotel found
west island - nearest hotel: Cocos Village Bungalows
fale old settlement - nearest hotel: No hotel found
ribeira grande - nearest hotel: Hospedaria JSF
albany - nearest hotel: No hotel found
al qadarif - nearest hotel: No hotel found
acarau - nearest hotel: No hotel found
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
rongelap - nearest hotel: No hotel found
seminole - nearest hotel: Best Western Plus Sanford
hirara - nearest hotel: 古謝そば屋
georgetown - nearest hotel: Page 63 hostel
avarua - nearest hotel: Paradise Inn
chinde - nearest hotel: Kanyum cottage village
hawaiian paradise park - nearest hotel: No

,City,Country,Lat,Lng,Humidity,Hotel Name
2,twon-brass,NG,4.3151,6.2418,82,KC Hotel
4,lihue,US,21.9789,-159.3672,67,Kauai Palms
8,kurchatov,RU,51.6601,35.6521,71,Россия
10,klyavlino,RU,54.2649,52.0267,80,No hotel found
14,adamstown,PN,-25.0660,-130.1015,80,No hotel found
...,...,...,...,...,...,...
562,tsiombe,MG,-25.3000,45.4833,72,No hotel found
563,talodi,SD,10.6325,30.3797,69,No hotel found
564,hubynykha,UA,48.8108,35.2547,61,No hotel found
565,tirano,IT,46.2154,10.1679,89,Bernina


In [13]:
# Configure the map
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    
    tiles = "OSM",
    color = "green",
    alpha = 0.5,

    frame_width = 800,
    hover_cols=['City', 'Country', 'Humidity', 'Cloudiness', 'Hotel Name'],
    frame_height = 600  
      
)

# Display the map plot
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Humidity,Hotel Name)